In [8]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

data = pd.read_csv("/workspaces/bakery_sales_prediction/0_DataPreparation/00_data/data_long.csv")
#print(data.head())
#data brot:
df= data[data["Warengruppe_1"] == 1].copy()
data_wetter = pd.read_csv("/workspaces/bakery_sales_prediction/2_BaselineModel/02_Franz/weather_data_with_flags.csv")

#Werktag
df['Werktag'] = (
    df[['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag']].sum(axis=1) > 0
) & (df['Feiertag'] == 0)
# In 0/1 umwandeln
df['Werktag'] = df['Werktag'].astype(int)

# Sicherstellen, dass "Datum" im Datumsformat ist
df['Datum'] = pd.to_datetime(df['Datum'])
# Jahreszeiten zuweisen
df['Frühling'] = df['Datum'].dt.month.isin([3, 4, 5]).astype(int)
df['Sommer'] = df['Datum'].dt.month.isin([6, 7, 8]).astype(int)
df['Herbst'] = df['Datum'].dt.month.isin([9, 10, 11]).astype(int)
df['Winter'] = df['Datum'].dt.month.isin([12, 1, 2]).astype(int)

#Number of Ships skalieren
# Min-Max-Skalierung manuell berechnen
df['Number_of_ships_scaled'] = (
    df['Number_of_ships'] - df['Number_of_ships'].min()
) / (df['Number_of_ships'].max() - df['Number_of_ships'].min())

#Bewölkung zsmfassen
df['Cloud_ok'] = (
    df[['Cloud_Clear', 'Cloud_Partly_Cloudy']].sum(axis=1) > 0)
# In 0/1 umwandeln
df['Cloud_ok'] = df['Cloud_ok'].astype(int)

#wind zsmfassen
df['Wind_ok'] = (
    df[['Wind_Light', 'Wind_Moderate']].sum(axis=1) > 0)
# In 0/1 umwandeln
df['Wind_ok'] = df['Wind_ok'].astype(int)

#neue Wetterkategorien
data_wetter["date"] = pd.to_datetime(data_wetter["date"])
df = df.merge(data_wetter, how="left", left_on="Datum", right_on="date").drop(columns=["date"])


df = df.drop(columns=["Warengruppe_1", "Warengruppe_2", "Warengruppe_3", "Warengruppe_4", "Warengruppe_5", "Warengruppe_6"])
df = df.drop(columns=["Datum"])
print(df)



          Umsatz  Temp_Very_Cold  Temp_Cold  Temp_Mild  Temp_Warm  Temp_Hot  \
0     148.828353               0          0          1          0         0   
1     159.793757               0          0          1          0         0   
2     111.885594               0          0          0          1         0   
3     168.864941               0          0          1          0         0   
4     171.280754               0          0          1          0         0   
...          ...             ...        ...        ...        ...       ...   
1814  198.225523               0          0          0          0         1   
1815  214.470393               0          0          0          0         1   
1816  129.801991               0          0          0          1         0   
1817  148.541340               0          0          0          0         1   
1818  123.075682               0          0          0          0         1   

      Cloud_Clear  Cloud_Partly_Cloudy  Cloud_Cloud

In [9]:
# 1. Schritt 1: Untersuche lineare Korrelation zwischen den einzelnen Features mit der Zielvariable
# mit Korrelationsmatrix
import pandas as pd

# Korrelationsmatrix erstellen
correlation_matrix = df.corr()

# Korrelation mit der Zielvariable
correlation_with_target = correlation_matrix["Umsatz"].sort_values(ascending=False)
print(correlation_with_target)




Umsatz                    1.000000
Ostertag                  0.343797
is_holiday                0.261852
Samstag                   0.261361
Sommer                    0.244808
Werktag                   0.188045
Silvester                 0.144596
Donnerstag                0.136544
Temp_Mild                 0.119302
Montag                    0.110951
wetter_sehr_schön_x       0.103739
wetter_sehr_schön_y       0.103585
Number_of_ships           0.103427
Number_of_ships_scaled    0.103427
Temp_Warm                 0.097649
Cloud_ok                  0.075294
Temp_Hot                  0.072790
Ship                      0.072114
VPI                       0.060790
Freitag                   0.056900
Heimspiel                 0.056840
KielerWoche               0.054958
Cloud_Partly_Cloudy       0.053832
Cloud_Clear               0.034175
Wind_Strong               0.022024
Weather_Moderate          0.021309
Wind_Moderate             0.011906
Weather_Good              0.006890
Weather_Severe      

In [10]:
from sklearn.feature_selection import mutual_info_regression

# Nicht-lineare Zusammenhänge analysieren
X = df.drop("Umsatz", axis=1)
y = df["Umsatz"]
mi = mutual_info_regression(X, y)
mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
print(mi_series)


Sonntag                   0.176498
Werktag                   0.089835
VPI                       0.081322
Sommer                    0.043375
Temp_Cold                 0.042691
Winter                    0.042234
is_holiday                0.035422
Frühling                  0.025158
Ostertag                  0.023765
Samstag                   0.023546
Cloud_Cloudy              0.020502
Cloud_ok                  0.020502
Temp_Hot                  0.017245
Temp_Mild                 0.016599
Montag                    0.016596
Cloud_Clear               0.015799
Freitag                   0.015384
Weather_Moderate          0.015052
Dienstag                  0.014640
Mittwoch                  0.014132
Weihnachtsmarkt           0.013634
wetter_sehr_schön_y       0.012352
wetter_sehr_schön_x       0.011700
Feiertag                  0.010256
Number_of_ships           0.009681
Number_of_ships_scaled    0.009438
Weather_Light_Issues      0.008585
Temp_Very_Cold            0.007458
Silvester           

In [11]:

import numpy as np

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Stark korrelierte Features (> 0.5) finden
high_corr = corr_matrix.abs() > 0.7
np.fill_diagonal(high_corr.values, False)  # Hauptdiagonale ignorieren
correlated_features = [column for column in high_corr.columns if any(high_corr[column])]
print(f"Stark korrelierte Features:", correlated_features)

Stark korrelierte Features: ['Cloud_Cloudy', 'Wind_Light', 'Wind_Moderate', 'Wind_Strong', 'Number_of_ships', 'Ship', 'wetter_sehr_schön_x', 'wetter_sehr_schlecht_x', 'Number_of_ships_scaled', 'Cloud_ok', 'Wind_ok', 'wetter_sehr_schön_y', 'wetter_sehr_schlecht_y']


In [12]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF für alle Features berechnen
X = df.drop("Umsatz", axis=1)
vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)



/workspaces/bakery_sales_prediction/.venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                   Feature           VIF
0           Temp_Very_Cold  5.967128e+08
1                Temp_Cold           inf
2                Temp_Mild           inf
3                Temp_Warm           inf
4                 Temp_Hot           inf
5              Cloud_Clear  1.394863e+05
6      Cloud_Partly_Cloudy           inf
7             Cloud_Cloudy           inf
8               Wind_Light           inf
9            Wind_Moderate  1.937751e+05
10             Wind_Strong           inf
11            Weather_Good           inf
12    Weather_Light_Issues           inf
13        Weather_Moderate           inf
14          Weather_Severe           inf
15             KielerWoche  1.167711e+00
16                  Montag           inf
17                Dienstag           inf
18                Mittwoch           inf
19              Donnerstag           inf
20                 Freitag           inf
21                 Samstag           inf
22                 Sonntag           inf
23              

In [13]:
# suche miteinander korrelierrende features

In [14]:

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Schwellenwert für "stark korreliert"
threshold = 0.4

# Finde die Paare mit starker Korrelation
strong_corr_pairs = set()

# Iteriere über die obere Dreiecksmatrix der Korrelationsmatrix
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:  # Korrelation über dem Schwellenwert
            strong_corr_pairs.add((corr_matrix.columns[i], corr_matrix.columns[j]))

# Anzeigen der Paare starker Korrelationen
print("Stark korrelierte Feature-Paare:")
print(strong_corr_pairs)


Stark korrelierte Feature-Paare:
{('Winter', 'Temp_Mild'), ('Sonntag', 'Umsatz'), ('wetter_sehr_schön_x', 'Temp_Hot'), ('Weather_Light_Issues', 'Weather_Good'), ('Wind_Moderate', 'Wind_Light'), ('Heimspiel', 'Samstag'), ('wetter_sehr_schön_y', 'wetter_sehr_schön_x'), ('Number_of_ships_scaled', 'Ship'), ('Werktag', 'Sonntag'), ('Number_of_ships_scaled', 'Number_of_ships'), ('Winter', 'Temp_Cold'), ('wetter_sehr_schlecht_y', 'wetter_sehr_schlecht_x'), ('Wind_Strong', 'Wind_Moderate'), ('Cloud_ok', 'Cloud_Partly_Cloudy'), ('Temp_Mild', 'Temp_Cold'), ('Sommer', 'Temp_Cold'), ('Wind_ok', 'Wind_Moderate'), ('wetter_sehr_schön_y', 'Temp_Hot'), ('Cloud_Cloudy', 'Cloud_Partly_Cloudy'), ('Weather_Moderate', 'Weather_Light_Issues'), ('Cloud_ok', 'Cloud_Cloudy'), ('Cloud_Cloudy', 'Cloud_Clear'), ('Cloud_ok', 'Cloud_Clear'), ('Ship', 'Number_of_ships'), ('Werktag', 'Samstag'), ('Sommer', 'Temp_Warm'), ('Wind_ok', 'Wind_Strong')}
